In [ ]:
######################## BLACK SCHOLES Project Code ########################

############## All Python code was written in Jupiter Notebook ##############

# Import needed libraries
import pandas as pd
import pandas_datareader.data as pdr
import datetime as dt
import numpy as np
import scipy.stats as si
import matplotlib.pyplot as plt
from matplotlib import style
import yfinance as yf
%matplotlib inline

######################## Bollinger Band Code ########################


# Get Data for SPY from Yahoo Finance
yf.pdr_override()
data = pdr.get_data_yahoo('SPY', start = '2009-12-04', end='2015-01-01')
SPY = data

# Iterate over YF Data to get Bollinger Bands needed values
for item in(SPY):
    #Moving average calculation
    SPY['30 Day MA'] = SPY['Adj Close'].rolling(window=20).mean()
    
    #Find standard deviation of previous 20 days in the market
    SPY['30 Day STD'] = SPY['Adj Close'].rolling(window=20).std() 
    
    #Find the upper and lower bounds of 2 standard deviations away from the moving average for Bollinger Band
    SPY['Upper Band'] = SPY['30 Day MA'] + (SPY['30 Day STD'] * 2)
    SPY['Lower Band'] = SPY['30 Day MA'] - (SPY['30 Day STD'] * 2)



# Get new data from Yahoo Finance 
data1 = pdr.get_data_yahoo('SPY', start = '2010-01-04', end='2015-01-01')
SPY1 = data1


#create new data frame by inner joining (merging) on the 'Date' column in each dataframe
SPYdf = SPY.merge(SPY1, left_on='Date', right_on='Date')


# Set new Signal cell to buy (1) when the the previous Adj Close price is less than the current day's
# and if the Adj Close price is less than the upper band of the Bollinger
# Set the Signal cell to not buy (0) otherwise
x=[]
for row in range(len(SPYdf)):
    if (SPYdf['Adj Close_x'].iloc[row-1] < SPYdf['Adj Close_x'].iloc[row]) and SPYdf['Adj Close_x'].iloc[row] < SPYdf['Upper Band'].iloc[row]:
                x.append(1)
    else:
                x.append(0)

# Set x equal to the created array
x = np.asarray(x)

# Assign each value of the array to the specific data in the dataframe in the new column 'Signal'
SPYdf['Signal'] = x


######################## Black Scholes Equation code ########################

BSdata= pdr.get_data_yahoo('SPY ^TNX ^VIX', start = '2010-01-04', end = '2015-01-01')

# Create function for the Scholes Equation
def blackScholes_CALL(S, K, T, r, Sig):
    d1 = (np.log(S / K) + (r + 0.5 * Sig**2) * T) / (Sig * np.sqrt(T))
    #S = spot(current stock or underlying price)
    #K = strike
    #T = time to mature
    #r = risk free interest rate
    #Sig = volatility of asset
    d2 = d1 - (Sig * np.sqrt(T))

    price = (S * si.norm.cdf(d1, 0.0, 1.0) - K * np.exp(-r * T) * si.norm.cdf(d2, 0.0, 1.0))
    return price

# Input the values from the BSdata for S, K, T, r, Sig
y = blackScholes_CALL(BSdata['Adj Close']['SPY'], (BSdata['Adj Close']['SPY'] + 2) , (7/360),
                      BSdata['Adj Close']['^TNX'], (BSdata['Adj Close']['^VIX'] / 100))

for item in(SPYdf):
    SPYdf['Signal'] = x
    SPYdf['Option Price'] = y

######################## Difference in 7 Day Stock Price ########################  

# Find the 7 day difference for the Adjust Close prices of SPY
data3 = pdr.get_data_yahoo('SPY', start = '2010-1-1', end='2015-1-13')
SPYdiff = data3

# Iterate over each day and calculate the 7 day difference in the stock price
difference_data = []
for row in range(len(SPYdiff)-7):
    q = SPYdiff['Adj Close'].iloc[row+7] - SPYdiff['Adj Close'].iloc[row]
    difference_data.append(q)

# Create an array out of the for loop
difference_data = np.asarray(difference_data)

# Add the newly created array as a column in the SPY dataframe   
for item in(SPYdf):
    SPYdf['7 Day Difference'] = difference_data

######################## Estimate of Profit or Loss of the Option ########################

# Find the estimates 
pl_estimate = []
for row in range(len(SPYdf)):
    if SPYdf['Signal'].iloc[row] == 1:
        a = SPYdf['7 Day Difference'].iloc[row]-SPYdf['Option Price'].iloc[row]-2
        if a < -SPYdf['Option Price'].iloc[row]:
            a = -SPYdf['Option Price'].iloc[row]
        pl_estimate.append(a)
    else: 
        pl_estimate.append(0)

# Create an array out of the For-loop for the estimate
pl_estimate = np.asarray(pl_estimate)

# Add the newly created array as a column in the SPY dataframe   
for item in(SPYdf):
    SPYdf['P/L Estimate'] = pl_estimate
    
print(SPYdf)

######################## Estimate of Profit or Loss of the Option ########################

# Plot the Profit/Loss Estimate of the Option and the Black Scholes Option Price
SPYdf[['P/L Estimate', 'Option Price']].plot(figsize=(12,6))
plt.title('Profit/Loss Estimates of the Options 2010-2015')
plt.ylabel('Price (USD)')
plt.show();

#Plot 30 Day Bollinger Band for SPY (2010-2015)
SPYdf[['Adj Close_x', '30 Day MA', 'Upper Band', 'Lower Band']].plot(figsize=(12,6))
plt.title('30 Day Bollinger Band for SPY 2010-2015')
plt.ylabel('Price (USD)')
plt.show();

